In [2]:
#Cargamos las librerías
import numpy as np
!pip install bokeh
import bokeh.plotting as bkp
import bokeh.models as bkm
import pandas as pd
import datetime as dt


/bin/sh: pip: command not found


In [4]:
#Obtenemos los datos de anomalías de temperaturas de la página de NOAA (descargada en la carpeta)
#https://www.ncdc.noaa.gov/monitoring-references/faq/anomalies.php
#Usamos la 5 línea como encabezados (header=4) de las columnas, y leemos el archivo a partir de ahí
temp_anomalia=pd.read_csv("https://www.ncdc.noaa.gov/cag/time-series/global/globe/land_ocean/p12/12/1880-2017.csv", sep=",", header=4,comment='#')

In [5]:
#Leemos las primeras 5 filas para ver que se han cargado bien los datos
temp_anomalia.head(n=5)

,Year,Value
0,188001,0.00
1,188002,-0.12
2,188003,-0.13
3,188004,-0.05
4,188005,-0.07


In [6]:
#renombramos los encabezados
temp_anomalia.set_axis(['fecha','anomalia'], axis=1, inplace=True)

In [7]:
#transformamos la fecha al formato estándar
temp_anomalia['fecha']=pd.to_datetime(temp_anomalia['fecha'],format="%Y%m")

In [8]:
#Como los datos son mensuales, calculamos el promedio anual (La forma de hacerlo es una convolución con 
#un vector con 12 unos. Posteriormente habría que dividir entre 12, excepto en los bordes, que la convolución toma
#menos datos. Para saber entre que número hay que dividir, uso una columna auxiliar llena de unos a la cual también
#realizo la misma convolución). Suena raro, pero es escueto de programar.

window_size = 12
window = np.ones(window_size)
columna_aux = np.ones(len(temp_anomalia['fecha']))
temp_anomalia['media_anual']=np.convolve(temp_anomalia['anomalia'], window, 'same')/np.convolve(columna_aux, window, 'same')

In [9]:
#Ahora calculamos el promedio a 10 años con el mismo procedimiento.
window_size = 120
window = np.ones(window_size)
columna_aux = np.ones(len(temp_anomalia['fecha']))
temp_anomalia['media_10años']=np.convolve(temp_anomalia['anomalia'], window, 'same')/np.convolve(columna_aux, window, 'same')

In [10]:
#Creamos una figura de Bokeh que va a contener nuestra gráfica. Indico el tamaño, que el eje X van a ser fechas
#y las herramientas activas (mover la gráfica y hacer zoom con el ratón)

p = bkp.figure(tools=["pan,wheel_zoom,box_zoom,save,reset"],
               width=900, height=500, x_axis_type="datetime", active_drag = "pan", active_scroll = "wheel_zoom")

#Represento los datos mensuales como puntos rojos
p.circle(temp_anomalia['fecha'], temp_anomalia['anomalia'], size=4, color='red', alpha=0.4, legend='Datos Mensuales')

#Represento el promedio anual como una línea azul
p.line(temp_anomalia['fecha'], temp_anomalia['media_anual'], color='navy', alpha=0.7, 
       legend='Promedio anual',line_width=3)

#Represento el promedio a 10 años como una línea negra
p.line(temp_anomalia['fecha'], temp_anomalia['media_10años'], color='black', alpha=0.8, 
       legend='Promedio 10 años',line_width=3)

GlyphRenderer(id='0f9fba97-e746-4906-87aa-6feede13d126', ...)

In [11]:
#Indico las propiedades del gráfico
p.title.text = "Anomalía de Temperatura Respecto a la Media entre 1901-2000"
p.title.text_color = "black"
p.title.text_font = "garamond"
p.title.text_font_style = "bold"
p.title.text_font_size = "16pt"
p.title.align = "center"

p.legend.location = "top_left"

p.background_fill_color = "olive"
p.background_fill_alpha = 0.1

p.xaxis.axis_label = 'Fecha'
p.xaxis.axis_label_text_font_style = "italic"
p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label = 'Anomalía ºC'
p.yaxis.axis_label_text_font_style  = "italic"
p.yaxis.axis_label_text_font_size = "14pt"

p.ygrid.grid_line_color="blue"
p.ygrid.grid_line_alpha=0.1
p.xgrid.grid_line_color="blue"
p.xgrid.grid_line_alpha=0.1

p.yaxis.major_tick_line_color = "red"
p.yaxis.major_tick_line_width = 3
p.yaxis.minor_tick_line_color = "orange"

p.outline_line_width = 7
p.outline_line_alpha = 0.3
p.outline_line_color = "navy"


In [12]:
#Muestro el gráfico
bkp.output_notebook()
bkp.show(p)

Loading BokehJS ...